# Estimation of failure probability due to wave run-up (Hunt's equation)

In this example, we will demonstrate the application of various reliability methods (`directional_sampling`, `importance_sampling`, `adaptive_importance_sampling`, `subset_simulation`) to estimate the probability of failure of a levee due to wave run-up. The failure mechanism is characterized by the Hunt's equation, which is a simple model to estimate the wave run-up.

### Define model

First, we import the necessary packages:

In [5]:
from streams import ReliabilityProject, DistributionType, ReliabilityMethod, StandardNormal, SampleMethod
import numpy as np

We consider the following limit state function:

$Z = h_{crest} - (h + R_u)$

where: <br>

$h_{crest}$ is the crest height of the flood defence (m) <br>
$h$ is the water level on the flood defence (m) <br>
$R_u$ represents the wave run-up (m)

The wave run-up $R_u$ is derived using the Hunt's equation:

$R_u = \xi \cdot H_s$

with:

$\xi = \frac{\tan (\alpha)}{\sqrt{2\cdot \pi \cdot H_s/L_0}} $

$L_0 = g \cdot T_p^2$

where:

$H_s$ is the significant wave height (m) <br>
$T_p$ is the peak wave period (s) <br>
$g$ is the gravitational acceleration ($9.81 m/s ^2$) <br>
$\tan (\alpha)$ is the slope of the flood defence (-)

The parameters $h$, $H_s$ and $T_p$ represent the imposed load, while $h_{crest}$ and $\tan (\alpha)$ stand for the strength of the levee.

In [6]:
def hunt(t_p, tan_alpha, h_s, h_crest, h):
    g = 9.81
    l_0 = g  * t_p * t_p
    xi = tan_alpha / np.sqrt(2 * np.pi * h_s / l_0)
    r_u = xi * h_s

    return h_crest - (h + r_u)

To perform a reliability analysis, we create a reliability project and specify the limit state function (model):

In [7]:
project = ReliabilityProject()
project.model = hunt

We assume the following distributions for the parameters present in the limit state function:

In [8]:
project.variables["t_p"].distribution = DistributionType.log_normal
project.variables["t_p"].mean = 6
project.variables["t_p"].deviation = 2

project.variables["tan_alpha"].distribution = DistributionType.deterministic
project.variables["tan_alpha"].mean = 0.333333

project.variables["h_s"].distribution = DistributionType.log_normal
project.variables["h_s"].mean = 3
project.variables["h_s"].deviation = 1

project.variables["h_crest"].distribution = DistributionType.log_normal
project.variables["h_crest"].mean = 10
project.variables["h_crest"].deviation = 0.05

project.variables["h"].distribution = DistributionType.exponential
project.variables["h"].shift = 0.5
project.variables["h"].scale = 1

### Perform reliability calculations with Directional Sampling

We start with the reliability method `directional_sampling`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.design_point`.

In [9]:
project.settings.reliability_method = ReliabilityMethod.directional_sampling
project.settings.minimum_directions = 10000
project.settings.maximum_directions = 20000
project.settings.variation_coefficient = 0.02

project.run()

def read_results(dp):

    beta = dp.reliability_index

    print(f"Beta = {beta}")

    pf = StandardNormal.get_q_from_u(beta)
    print(f"Probability of failure = {pf}")

    for alpha in dp.alphas:
        print(f"{alpha.variable.name}: alpha = {alpha.alpha}, x = {alpha.x}")

    if dp.is_converged:
        print(f"Converged (convergence = {dp.convergence} < {project.settings.variation_coefficient})")
    else:
        print(f"Not converged (convergence = {dp.convergence} > {project.settings.variation_coefficient})")
        
    print(f"Model runs = {dp.total_model_runs}")

read_results(project.design_point)

Beta = 1.8711604829760657
Probability of failure = 0.030661420137601914
t_p: alpha = -0.7599460091669649, x = 9.030827950876963
tan_alpha: alpha = 0.0, x = 0.333333
h_s: alpha = -0.3881938534023303, x = 3.602786268469947
h_crest: alpha = 0.01326241797916879, x = 9.998634296961573
h: alpha = -0.5211637972856513, x = 2.3034158389094634
Converged (convergence = 0.019998996781194274 < 0.02)
Model runs = 82734


### Perform reliability calculations with Importance Sampling

We now conduct the reliability analysis using the `importance_sampling`.

In [10]:
project.settings.reliability_method = ReliabilityMethod.importance_sampling
project.settings.minimum_samples = 1000
project.settings.maximum_samples = 100000
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.878734663894228
Probability of failure = 0.030140366090895945
t_p: alpha = -0.7635028627497121, x = 9.06736272143939
tan_alpha: alpha = 0.0, x = 0.333333
h_s: alpha = -0.3800099068522848, x = 3.5882733586165823
h_crest: alpha = 0.018696192467041448, x = 9.99811893023878
h: alpha = -0.5218297630974563, x = 2.311251988352642
Not converged (convergence = 0.020664889091332777 > 0.02)
Model runs = 100001


### Perform reliability calculations with Adaptive Importance Sampling

We now conduct the reliability analysis using the `adaptive_importance_sampling` method.

In [11]:
project.settings.reliability_method = ReliabilityMethod.adaptive_importance_sampling
project.settings.minimum_samples = 10000
project.settings.maximum_samples = 100000
project.settings.minimum_variance_loops = 5
project.settings.maximum_variance_loops = 10
project.settings.fraction_failed = 0.5
project.settings.variation_coefficient = 0.02

project.run()

read_results(project.design_point)

Beta = 1.8800015664915952
Probability of failure = 0.030053932215033417
t_p: alpha = -0.7655055789023627, x = 9.08130180954658
tan_alpha: alpha = 0.0, x = 0.333333
h_s: alpha = -0.38014700983128114, x = 3.5891344163464347
h_crest: alpha = 0.008789833715386889, x = 9.999048806910018
h: alpha = -0.519049321749822, x = 2.3043679996051454
Converged (convergence = 0.019998432841262077 < 0.02)
Model runs = 65024


### Perform reliability calculations with Subset Simulation

We now conduct the reliability analysis using the `subset_simulation` method.

In [12]:
project.settings.reliability_method = ReliabilityMethod.subset_simulation
project.settings.minimum_samples = 1000
project.settings.maximum_samples = 50000
project.settings.variation_coefficient = 0.02
project.settings.sample_method = SampleMethod.adaptive_conditional

project.run()

read_results(project.design_point)

Beta = 1.8810876257425375
Probability of failure = 0.029980000000000024
t_p: alpha = -0.7686904565976026, x = 9.101434711035957
tan_alpha: alpha = 0.0, x = 0.333333
h_s: alpha = -0.3831936176465498, x = 3.5962991283918466
h_crest: alpha = 0.019079768958409998, x = 9.998080661130349
h: alpha = -0.5117749463848468, x = 2.2846805335478786
Not converged (convergence = 0.006834554905808205 > 0.02)
Model runs = 45000
